In [1]:
!pip install rank_bm25 transformers datasets torch
!pip install rouge_score # Değerlendirme için

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=e15d63d3b572503889b199b27f376f17c25dee16fe5970c72e1be8af8c27c3dc
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [ ]:
from datasets import load_dataset

# SQuAD dataset: Includes Wikipedia paragraphs and questions asked to them
dataset = load_dataset("squad", split="validation[:1000]") # First 1000 examples for speed up

# Create Corpus
# 'context' (paragraph) in every example is a document
# Clean up repetative documents using set
corpus = list(set(dataset['context']))

print(f"Count of documents in library: {len(corpus)}")

Veri seti indiriliyor...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

plain_text/validation-00000-of-00001.par(…):   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Kütüphanedeki Belge Sayısı: 93
Örnek Belge: By July 1944, the Red Army was deep into Polish territory and pursuing the Germans toward Warsaw. Kn...


In [ ]:
from rank_bm25 import BM25Okapi
import string

# A simple tokenizer (Word seperator)
def simple_tokenizer(text):
    # Convert to lowercase, erease punctuation and split
    text = text.lower().translate(str.maketrans('', '', string.punctuation))
    return text.split()


tokenized_corpus = [simple_tokenizer(doc) for doc in corpus]

bm25 = BM25Okapi(tokenized_corpus)


# --- TESTING ---
query = "What creates a magnetic field?"
tokenized_query = simple_tokenizer(query)
doc_scores = bm25.get_scores(tokenized_query)
best_docs = bm25.get_top_n(tokenized_query, corpus, n=1)

print(f"\nQuestion: {query}")
print(f"Found Document: {best_docs[0]}")

Belgeler indeksleniyor (BM25)...
Arama motoru hazır!

Soru: What creates a magnetic field?
Bulunan Belge: Concerns were raised over whether Levi's Stadium's field was of a high enough quality to host a Super Bowl; during the inaugural season, the field had to be re-sodded multiple times due to various issues, and during a week 6 game earlier in the 2015 season, a portion of the turf collapsed under Baltimore Ravens kicker Justin Tucker, causing him to slip and miss a field goal, although the field has not had any major issues since. As is customary for Super Bowl games played at natural grass stadiums, the NFL re-sodded the field with a new playing surface; a hybrid Bermuda 419 turf. NFL and Atlanta Braves field director Ed Mangan stated that the field was in "great shape" for gameday. However, the turf showed problem throughout the game, with a number of players needing to change their cleats during the game and player slipping during plays all throughout the game.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_name = "google/flan-t5-small" 
print(f"model name: {model_name} ")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

google/flan-t5-small modeli yükleniyor...


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Generator modeli hazır!


In [ ]:
def rag_pipeline(question, n_docs=1):
    # 1. STEP: Retrieval
    tokenized_query = simple_tokenizer(question)
    # Bring most related 'n' documents
    retrieved_docs = bm25.get_top_n(tokenized_query, corpus, n=n_docs)
    context = " ".join(retrieved_docs) # Concatanate documents

    # 2. STEP: Prompt Preperation (Generation Input)
    input_text = f"Answer the question based on the context.\nContext: {context}\nQuestion: {question}"

    # 3. STEP: Give answer (Generation)
    input_ids = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512).input_ids.to(device)

    outputs = model.generate(input_ids, max_length=64, num_beams=4, early_stopping=True)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return answer, context

# --- TEST ---
question = "Which team won the Super Bowl 50?"
answer, source = rag_pipeline(question)

print(f"QUESTION: {question}")
print(f"ANSWER (RAG): {answer}")
print(f"USED SOURCE: {source[:200]}...")

SORU: Which team won the Super Bowl 50?
CEVAP (RAG): 49ers
KULLANILAN KAYNAK (Kısaltılmış): The league announced on October 16, 2012, that the two finalists were Sun Life Stadium and Levi's Stadium. The South Florida/Miami area has previously hosted the event 10 times (tied for most with New...


In [6]:
!pip install evaluate rouge_score bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.5 MB/s eta 0:00:00


In [ ]:
import evaluate
import numpy as np
from tqdm import tqdm

# Upload Metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

# Test Settings
TEST_SIZE = 20 # Only 20 questions for speed up
test_data = dataset.select(range(TEST_SIZE))


predictions = []
references = []
retrieval_hits = 0 # Num of correct documents

# Evaluation Loop
for item in tqdm(test_data):
    question = item['question']
    true_answer = item['answers']['text'][0] # Reel answer
    true_context = item['context']           # Reel source text

    # RAG System
    # n_docs=1, so it returns most related 1 document
    pred_answer, retrieved_context = rag_pipeline(question, n_docs=1)

    # Append to Lists (to measure Generation)
    predictions.append(pred_answer)
    references.append(true_answer)

    # Retrieval Success (Recall@1)
    # Is the found document a piece from real text or does it match perfectly?
    # Manual control is needed
    if true_context[:100] in retrieved_context: # If first 100 characters are accurate then it is correct
        retrieval_hits += 1

# Calculate scores

# Retrieval Score
recall_at_1 = retrieval_hits / TEST_SIZE

# Generation Scores
bleu_result = bleu.compute(predictions=predictions, references=references)
rouge_result = rouge.compute(predictions=predictions, references=references)

print("\n--- Calculating BERTScore ---")
bert_result = bertscore.compute(predictions=predictions, references=references, lang="en")

# Reporting
print("\n" + "="*30)
print("   RAG SYSTEM")
print("="*30)
print(f"🔍 Retrieval Success:")
print(f"   Recall@1: {recall_at_1:.2%} (Portion of finding right document)")
print("-" * 30)
print(f"📝 Generation Quality:")
print(f"   BLEU Score: {bleu_result['bleu']:.4f}")
print(f"   ROUGE-L   : {rouge_result['rougeL']:.4f}")
print(f"   BERTScore : {np.mean(bert_result['f1']):.4f}")
print("="*30)

20 soru üzerinde değerlendirme başlıyor...


100%|██████████| 20/20 [00:03<00:00,  5.45it/s]



--- BERTScore Hesaplanıyor (Biraz sürebilir) ---


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



   RAG SİSTEMİ KARNESİ
🔍 Retrieval (Erişim) Başarısı:
   Recall@1: 35.00% (Doğru belgeyi bulma oranı)
------------------------------
📝 Generation (Üretim) Kalitesi:
   BLEU Score: 0.0000
   ROUGE-L   : 0.3000
   BERTScore : 0.8812
